<a href="https://colab.research.google.com/github/lyeslehara1996/PFE/blob/main/Essay_correction_Profipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install np_utils

In [213]:
#declaration des module 

import pandas as pd 
import re 
import nltk

import matplotlib.pyplot as plt
from sklearn.preprocessing  import MinMaxScaler
import numpy as np
from keras.layers import Dropout

from sklearn.preprocessing import LabelBinarizer
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
import string as st
SAVEd = False

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,classification_report
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [214]:
from keras.utils.np_utils import to_categorical

In [215]:
df=pd.read_excel('/content/SemEval2017.xlsx')

netoyage de dataset



In [216]:
df.drop("Unnamed: 3", axis=1, inplace=True)
df.drop("Unnamed: 4", axis=1, inplace=True)
df.drop("Unnamed: 5", axis=1, inplace=True)
df.drop("Unnamed: 6", axis=1, inplace=True)
df.drop("Unnamed: 7", axis=1, inplace=True)
df.drop("Unnamed: 8", axis=1, inplace=True)
df.drop("Unnamed: 9", axis=1, inplace=True)
df.drop("Unnamed: 10", axis=1, inplace=True)
df.drop("Unnamed: 11", axis=1, inplace=True)
df.drop("Unnamed: 12", axis=1, inplace=True)
df.drop("Unnamed: 13", axis=1, inplace=True)
df.drop("Unnamed: 14", axis=1, inplace=True)
df.drop("Unnamed: 15", axis=1, inplace=True)
df.head()

,id,polariy,Comments
0,619950566786113024,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T..."
1,619969366986235008,neutral,Order Go Set a Watchman in store or through ou...
2,619971047195044992,negative,If these runway renovations at the airport pre...
3,619974445185302016,neutral,If you could ask an onstage interview question...
4,619987808317406976,positive,A portion of book sales from our Harper Lee/Go...


**Prétraitement**

transformer les mots en miniscule


In [217]:
df_clean = df
df_clean.Comments=df_clean.Comments.str.lower()
df_clean.head() 


,id,polariy,Comments
0,619950566786113024,neutral,"picturehouse's, pink floyd's, 'roger waters: t..."
1,619969366986235008,neutral,order go set a watchman in store or through ou...
2,619971047195044992,negative,if these runway renovations at the airport pre...
3,619974445185302016,neutral,if you could ask an onstage interview question...
4,619987808317406976,positive,a portion of book sales from our harper lee/go...


Suppression de tout les symbole  

In [218]:
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r'https?:\/\/\S+', ' ', str(x)))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', x))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r'{link}', ' ', x))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r'&[a-z]+;', ' ', str(x)))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r"[^a-z]", ' ', str(x)))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: re.sub(r'@mention', ' ', x))
df_clean['Comments'] = df_clean['Comments'].apply(lambda x: " ".join(x.lower() for x in str(x).split()  if len(x)>2 ))
df_clean

,id,polariy,Comments
0,619950566786113024,neutral,picturehouse pink floyd roger waters the walll...
1,619969366986235008,neutral,order set watchman store through our website b...
2,619971047195044992,negative,these runway renovations the airport prevent f...
3,619974445185302016,neutral,you could ask onstage interview question miss ...
4,619987808317406976,positive,portion book sales from our harper lee set wat...
...,...,...,...
20628,681877834982232064,neutral,shaquillehoneal from what think you asking ord...
20629,681879579129200000,positive,iran ranks liver surgeries allah bless the cou...
20630,681883903259357056,neutral,hours before arrived saudi arabia tuesday turk...
20631,681904976860327040,negative,vanityfair alex kim kardashian worth how love ...


In [219]:
df_clean['polariy'] = df_clean['polariy'].apply(lambda x: re.sub(r'https?:\/\/\S+><', ' ', str(x)))
df_clean['polariy']

0         neutral
1         neutral
2        negative
3         neutral
4        positive
           ...   
20628     neutral
20629    positive
20630     neutral
20631    negative
20632     neutral
Name: polariy, Length: 20633, dtype: object

In [195]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [220]:
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
df_clean['Comments']=df_clean['Comments'].apply(remove_stopwords)

{'our', 'too', 'myself', 'against', 'more', 'who', "that'll", 'were', 'has', 'have', "shan't", 'into', 'you', 'am', 'doesn', 'a', "couldn't", "haven't", 'this', 'from', "should've", 'up', 'hers', 'been', 'below', 'to', 'hasn', 'was', 'there', 'an', 'shan', 'such', "you'll", 'when', 'of', 'by', 'with', "don't", 'again', 'same', 'about', 'shouldn', 'most', 'did', 'is', 'mightn', 'his', 'mustn', 've', 'so', 'just', 'should', 'whom', 'yourself', 'yourselves', 'him', 'll', 'above', "won't", 'that', 'under', 'couldn', 'those', 'we', 'here', 'as', "isn't", 'or', 'and', 'ours', "mightn't", 'over', "didn't", 'needn', 'itself', 'be', 'than', 'y', 'ma', 'then', 'won', 'aren', 'some', 's', "hadn't", 'm', 'while', 'he', 'they', 'only', 'these', 'down', 'not', 'her', "you're", "you've", 're', 'themselves', 'wasn', "she's", 'theirs', "aren't", 'few', 't', 'all', 'because', "shouldn't", 'do', 'yours', "hasn't", "it's", 'didn', 'weren', 'are', 'out', 'o', 'my', 'having', 'own', 'doing', 'ourselves', "w

In [258]:
reviews =  df_clean['Comments']
labels =  df_clean['polariy']
reviews

0        picturehouse pink floyd roger waters walll ope...
1        order set watchman store website tuesday get h...
2        runway renovations airport prevent seeing tayl...
3        could ask onstage interview question miss usa ...
4        portion book sales harper lee set watchman rel...
                               ...                        
20628    shaquillehoneal think asking order future drak...
20629       iran ranks liver surgeries allah bless country
20630    hours arrived saudi arabia tuesday turkish pre...
20631    vanityfair alex kim kardashian worth love kim ...
20632    guess even pandora knows justin bieber grown c...
Name: Comments, Length: 20633, dtype: object

In [259]:
reviews

0        picturehouse pink floyd roger waters walll ope...
1        order set watchman store website tuesday get h...
2        runway renovations airport prevent seeing tayl...
3        could ask onstage interview question miss usa ...
4        portion book sales harper lee set watchman rel...
                               ...                        
20628    shaquillehoneal think asking order future drak...
20629       iran ranks liver surgeries allah bless country
20630    hours arrived saudi arabia tuesday turkish pre...
20631    vanityfair alex kim kardashian worth love kim ...
20632    guess even pandora knows justin bieber grown c...
Name: Comments, Length: 20633, dtype: object

In [260]:
labels

0         neutral
1         neutral
2        negative
3         neutral
4        positive
           ...   
20628     neutral
20629    positive
20630     neutral
20631    negative
20632     neutral
Name: polariy, Length: 20633, dtype: object

In [261]:
revue_sans_ponctuation=[]

for sentence in reviews:
    revue_sans_ponctuation.append(' '.join(Word.strip(st.punctuation) for Word in sentence.split()))

In [262]:
reviews_cleaned = np.asarray(revue_sans_ponctuation)
reviews_cleaned

array(['picturehouse pink floyd roger waters walll opening sept making waves watch trailer rolling stone look',
       'order set watchman store website tuesday get half price gsaw gsawatchmanbook',
       'runway renovations airport prevent seeing taylor swift monday bad blood new meaning',
       ...,
       'hours arrived saudi arabia tuesday turkish president recep tayyip erdogan accused syria president mercilessly',
       'vanityfair alex kim kardashian worth love kim kardashian bad sun conure',
       'guess even pandora knows justin bieber grown condom played'],
      dtype='<U125')

In [263]:
review_array = np.asarray(reviews)
label_array = np.asarray(labels)

reviews_labels = np.stack((review_array, label_array), axis = 1)


In [264]:
list_index=[]

for i, text in enumerate(reviews_labels[:,0:1]):
    if(text == "\n"):
        list_index.append(i)

reviews_labels = np.delete(reviews_labels, list_index, axis=0)
reviews_labels

array([['picturehouse pink floyd roger waters walll opening sept making waves watch trailer rolling stone look',
        'neutral'],
       ['order set watchman store website tuesday get half price gsaw gsawatchmanbook',
        'neutral'],
       ['runway renovations airport prevent seeing taylor swift monday bad blood new meaning',
        'negative'],
       ...,
       ['hours arrived saudi arabia tuesday turkish president recep tayyip erdogan accused syria president mercilessly',
        'neutral'],
       ['vanityfair alex kim kardashian worth love kim kardashian bad sun conure',
        'negative'],
       ['guess even pandora knows justin bieber grown condom played',
        'neutral']], dtype=object)

In [294]:
encoder = LabelEncoder()
encoder.fit(label_array)
encoded_labels = encoder.transform(label_array)
# encoded_labels = to_categorical(encoded_labels)
encoded_labels

array([2, 2, 1, ..., 2, 1, 2])

In [295]:
review_train, review_test, label_train, label_test = train_test_split(reviews, encoded_labels,test_size=0.2, train_size=0.8, random_state=42)

print(review_train.shape, label_train.shape)
print(review_test.shape, label_test.shape)

(16506,) (16506,)
(4127,) (4127,)


In [296]:
tokenizer = Tokenizer(num_words=4000)
tokenizer.fit_on_texts(review_train)

review_train = tokenizer.texts_to_sequences(review_train)
review_test = tokenizer.texts_to_sequences(review_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

review_train = pad_sequences(review_train, padding='post', maxlen=maxlen)
review_test = pad_sequences(review_test, padding='post', maxlen=maxlen)

In [289]:
review_train


array([[132,  25,  27, ...,   0,   0,   0],
       [ 19,  99, 610, ...,   0,   0,   0],
       [376, 277, 156, ...,   0,   0,   0],
       ...,
       [125, 106, 613, ...,   0,   0,   0],
       [ 70,  49,  30, ...,   0,   0,   0],
       [ 11, 662,   8, ...,   0,   0,   0]], dtype=int32)

In [305]:

model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(maxlen,3)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [306]:
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 100, 50)           10800     
_________________________________________________________________
dropout_12 (Dropout)         (None, 100, 50)           0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 100, 50)           20200     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100, 50)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 100, 50)           20200     
_________________________________________________________________
dropout_14 (Dropout)         (None, 100, 50)           0         
Total params: 51,200
Trainable params: 51,200
Non-trainable params: 0
_________________________________________________

In [307]:
history=model.fit(review_train, label_train,batch_size=128, epochs=10, verbose=1,validation_data=(review_test, label_test))


Epoch 1/10


ValueError: ignored